In [5]:
import torch
from transformers import T5Tokenizer

In [7]:
model_name = "31_model_w_prefix_50_output.pt"
model = torch.load(f"models/{model_name}")
model.eval();
tokenizer = T5Tokenizer.from_pretrained("t5-base", model_max_length=1024)

In [33]:
def get_summary(text: str) -> str:
    text = "summarize: " + text
    encoded_text = tokenizer(text, return_tensors="pt")
    output_ids = model.generate(encoded_text.input_ids, 
                                attention_mask=encoded_text.attention_mask, 
                                max_length=128)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text

In [36]:
text = \
"""
Great upright lightweight hoover and good price. Suction power really good, plus lays flat when cleaning under furniture. Handheld section is also lightweight and bonus with the holder for the attachments that sits on the pole. I've only rated it 4 overall mainly because I do find I have to pull out all the hair from the vacuum head each use, to keep it working well. You also get a spare hepa filter with this. Battery life is pretty good, guess it depends how much time you spend vacuuming, but charges well. Overall would buy this again.
"""
text = text.replace('\n', '').strip()


In [37]:
summarized_text = get_summary(text)
len_original = len(text.split(' '))
len_summary = len(summarized_text.split(' '))

In [38]:
print(f"Original text: {len_original} words")
print(f"Summarized text: {len_summary} words")
print(f"Predicted summary: \n{summarized_text}")

Original text: 97 words
Summarized text: 31 words
Predicted summary: 
The upright vacuum is lightweight and has good suction power. It also has a holder for attachments. The battery life is good, but it charges well. Overall, would buy this again.
